In [1]:
mnist_results_paths = ['../output/natural-mnist-results.json',
                       '../output/ablation-mnist-results.json',
                       '../output/ablation-mnist-results2.json',
                       '../output/ablation-mnist-results3.json',
                       '../output/ablation-mnist-results-spsa.json']
cifar10_results_paths = ['../output/natural-cifar10-results.json',
                         '../output/natural-cifar10-results2.json',
                         '../output/ablation-cifar10-results.json',
                         '../output/ablation-cifar10-results2.json']

In [2]:
import pandas as pd
import numpy as np
from collections import OrderedDict
import re
import os
from scipy import stats

In [3]:
import json

def read_json(path):
    rows = []
    with open(path) as f:
        for line in f:
            if line.strip():
                row = json.loads(line[:-1])
                rows.append(row)
    df = pd.DataFrame(rows)
    if 'eps' in df.columns:
        df = df.drop_duplicates(['model_path', 'attack_type', 'eps'], keep='last')
    df = df.rename(columns={'attack': 'attack_type'})
    return df

def read_jsons(paths):
    dfs = [read_json(path) for path in paths if os.path.exists(path)]
    return pd.concat(dfs, sort=False).reset_index()

In [4]:
def process(df):
    df['attack_type'] = df.apply(
        lambda r: ('%s(eps=%.1f)' % (r['attack_type'], r['eps'])
                   if 'eps' in r else r['attack_type']), axis=1)
    df['model_name'] = df.model_path.apply(os.path.basename)
    counts = df.groupby(['model_name', 'attack_type']).agg({'accuracies': len})
    duplicates = counts[counts.accuracies >= 2]
    if len(duplicates) > 0:
        print(duplicates.reset_index().values)
    assert len(duplicates) == 0, 'Please remove duplicates'
    return df

# MNIST results

In [5]:
results = process(read_jsons(mnist_results_paths))

In [6]:
results.groupby('model_path').agg({'attack_type': len})

,attack_type
model_path,
/nfs/home2/minhle/newlogic/output/ablation-mnist-models/relog-elliptical-maxout_4-max_fit_l1_1.pkl,7
/nfs/home2/minhle/newlogic/output/ablation-mnist-models/relog-elliptical-maxout_4-sigmoid_out-max_fit_l1_1-overlay.pkl,7
/nfs/home2/minhle/newlogic/output/ablation-mnist-models/relog-elliptical-maxout_4-sigmoid_out-max_fit_l1_1-scrambling.pkl,7
/nfs/home2/minhle/newlogic/output/ablation-mnist-models/relog-elliptical-maxout_4-sigmoid_out-max_fit_l1_1.pkl,7
/nfs/home2/minhle/newlogic/output/ablation-mnist-models/relog-elliptical-maxout_4.pkl,7
/nfs/home2/minhle/newlogic/output/ablation-mnist-models/relog-maxout_4.pkl,7
/nfs/home2/minhle/newlogic/output/ablation-mnist-models/relog-minmaxout_2_4.pkl,7
/nfs/home2/minhle/newlogic/output/ablation-mnist-models/relog.pkl,7
/nfs/home2/minhle/newlogic/output/ablation-mnist-models/relu.pkl,7


In [7]:
def compute_accuracy_and_ttest(col):
    if col.name in ('index', 'model_name'):
        return list(col)
    formatted_accuracies = []
    for prev, curr in zip([None] + list(col), list(col)):
        if isinstance(curr, (str, float)):
            formatted_accuracies.append(curr)
        elif isinstance(curr, list):
            curr = np.around(curr, 2)
            acc = np.mean(curr)
            acc = '%.2f' % acc
            if isinstance(prev, list):
                prev = np.around(prev, 2)
                _, p_value = stats.ttest_ind(prev, curr, equal_var = False)
                acc += '*' if p_value < 0.05 else '<'
            else:
                acc += '<'
            formatted_accuracies.append(acc)
        else:
            raise ValueError("Unsupported type: " + str(curr))
    return pd.Series(formatted_accuracies)

In [8]:
def format_metrics(df, formatted_names, model_order, attack_order, values_col):
    # filtering
    df = df[df.model_name.isin(formatted_names) &
            df.attack_type.isin(attack_order)].copy()
    # convert to wide format
    df = df.pivot(index='model_name', values=values_col, columns='attack_type')
    # change column order
    df = (df.reindex(sorted(df.columns, key=attack_order.__getitem__), axis=1)
          .reset_index().fillna('-'))
    df['model_order'] = df.model_name.apply(model_order.__getitem__)
    df = df.sort_values(['model_order']).drop(columns='model_order')
    # compute mean accuracy and p-values
    df = df.apply(compute_accuracy_and_ttest)
    # format model names last
    df['model_name'] = df.model_name.apply(formatted_names.__getitem__)    
    return df

def format_table(df, formated_names, model_order, attack_order):
    df = format_metrics(df, formatted_names, model_order, attack_order, values_col='accuracies')
    # since the "sigmoid out" network always give 0.22% as max confidence (I shouldn't have
    # add sigmoid to the network), I decided to drop confidence measurement.
#     max_probs_table = format_metrics(results, formatted_names, model_order, attack_order, values_col='max_probs')
#     max_probs = (max_probs_table[['model_name', 'MaxConf(eps=0.3)', 'none(eps=nan)']]
#                  .rename(columns={'MaxConf(eps=0.3)': 'max_probs', 'none(eps=nan)': 'max_probs_natural'}))
#     df = pd.merge(accuracies_table, max_probs, on='model_name')
#     df['MaxConf(eps=0.3)'] = df.apply(lambda r: '%s (%s)' % (r['MaxConf(eps=0.3)'], r['max_probs']), axis=1)
#     df['none(eps=nan)'] = df.apply(lambda r: '%s (%s)' % (r['none(eps=nan)'], r['max_probs_natural']), axis=1)
#     df = df.drop(columns=['max_probs', 'max_probs_natural'])
    latex = df.to_latex()
    latex = latex.replace('*', '$^*$')
    latex = latex.replace('<', '\hspace{0.5em}')
    return df, latex

## Ablation table

In [9]:
formatted_names = OrderedDict([
    ('relu.pkl', 'Baseline'),
    ('relog.pkl', '+ ReLog'), 
    ('relog-maxout_4.pkl', '+ MaxOut (k=4)'),
    ('relog-minmaxout_2_4.pkl', '+ MinOut (k=2)'),
    ('relog-elliptical-maxout_4.pkl', '+ Elliptical'),
    ('relog-elliptical-maxout_4-max_fit_l1_1.pkl', '+ MaxFit'),  
    ('relog-elliptical-maxout_4-sigmoid_out-max_fit_l1_1.pkl', '+ MSE training'),  
    ('relog-elliptical-maxout_4-sigmoid_out-max_fit_l1_1-overlay.pkl', '+ Negative examples'),  
])
model_order = {n: i for i, n in enumerate(formatted_names)}

In [10]:
used_attacks = ['none(eps=nan)', # (natural)
                'FGM_inf(eps=0.3)', 'FGM_L2(eps=2.0)', 'CW(eps=nan)',
                'BIM(eps=0.3)', 'SPSA(eps=0.3)']
attack_order = {n: i for i, n in enumerate(used_attacks)}

In [11]:
results.attack_type.unique()

array(['none(eps=nan)', 'FGM_inf(eps=0.3)', 'FGM_L2(eps=2.0)',
       'CW(eps=nan)', 'BIM(eps=0.3)', 'SPSA(eps=0.3)', 'MaxConf(eps=0.3)'],
      dtype=object)

In [12]:
ablation_table, latex = format_table(results, formatted_names, model_order, attack_order)

In [13]:
print(latex)

\begin{tabular}{llllllll}
\toprule
attack\_type &           model\_name & none(eps=nan) & FGM\_inf(eps=0.3) & FGM\_L2(eps=2.0) & CW(eps=nan) & BIM(eps=0.3) & SPSA(eps=0.3) \\
\midrule
7 &             Baseline &         0.99\hspace{0.5em} &            0.33\hspace{0.5em} &           0.82\hspace{0.5em} &       0.05\hspace{0.5em} &        0.01\hspace{0.5em} &         0.36\hspace{0.5em} \\
6 &              + ReLog &         0.99\hspace{0.5em} &            0.64$^*$ &           0.92$^*$ &       0.42$^*$ &        0.15$^*$ &         0.52$^*$ \\
4 &       + MaxOut (k=4) &         0.98$^*$ &            0.56$^*$ &           0.89$^*$ &       0.10$^*$ &        0.07$^*$ &         0.52\hspace{0.5em} \\
5 &       + MinOut (k=2) &         0.98\hspace{0.5em} &            0.56\hspace{0.5em} &           0.90$^*$ &       0.09$^*$ &        0.12$^*$ &         0.60$^*$ \\
3 &         + Elliptical &         0.98\hspace{0.5em} &            0.70$^*$ &           0.89$^*$ &       0.37$^*$ &        0.17$^*$ &       

# CIFAR-10 results

In [14]:
cifar10_results = process(read_jsons(cifar10_results_paths))

In [15]:
cifar10_results.attack_type.unique()

array(['none(eps=nan)', 'FGM_L2(eps=2.0)', 'FGM_inf(eps=0.3)',
       'BIM(eps=0.3)', 'CW(eps=nan)'], dtype=object)

In [16]:
cifar10_results.model_name.unique()

array(['relu.pkl', 'relog.pkl', 'relog-maxout_4.pkl',
       'relog-minmaxout_2_4-max_fit_l2.pkl', 'relog-minmaxout_2_4.pkl',
       'relog-quadratic-max_fit_l2-maxout_4-sigmoid_out-overlay.pkl',
       'relog-quadratic-max_fit_l2-maxout_4-sigmoid_out-scrambling.pkl',
       'relog-quadratic-max_fit_l2-maxout_4-mse.pkl',
       'relog-quadratic-max_fit_l2-maxout_4-sigmoid_out.pkl',
       'relog-quadratic-max_fit_l2-maxout_4-mse-overlay.pkl',
       'relog-quadratic-max_fit_l2-maxout_4.pkl'], dtype=object)

In [17]:
formatted_names = OrderedDict([
    ('relu.pkl', 'Baseline'),
    ('relog.pkl', '+ ReLog'), 
    ('relog-maxout_4.pkl', '+ MaxOut (k=4)'),
    ('relog-minmaxout_2_4.pkl', '+ MinOut (k=2)'),
    ('relog-minmaxout_2_4-max_fit_l2.pkl', '+ MaxFit (L2=0.01)'),
    ('relog-quadratic-max_fit_l2-maxout_4.pkl', '+ Quadratic'),
    ('relog-quadratic-max_fit_l2-maxout_4-sigmoid_out.pkl', '+ MSE training'),
    ('relog-quadratic-max_fit_l2-maxout_4-sigmoid_out-overlay.pkl', '+ Negative examples')
])
model_order = {n: i for i, n in enumerate(formatted_names)}

In [18]:
ablation_table, latex = format_table(cifar10_results, formatted_names, model_order, attack_order)

In [19]:
print(latex)

\begin{tabular}{lllllll}
\toprule
attack\_type &           model\_name & none(eps=nan) & FGM\_inf(eps=0.3) & FGM\_L2(eps=2.0) & CW(eps=nan) & BIM(eps=0.3) \\
\midrule
7 &             Baseline &         0.87\hspace{0.5em} &            0.08\hspace{0.5em} &           0.28\hspace{0.5em} &       0.05\hspace{0.5em} &        0.00\hspace{0.5em} \\
6 &              + ReLog &         0.87\hspace{0.5em} &            0.09$^*$ &           0.30$^*$ &       0.05\hspace{0.5em} &        0.00\hspace{0.5em} \\
0 &       + MaxOut (k=4) &         0.87\hspace{0.5em} &            0.12$^*$ &           0.29$^*$ &       0.01$^*$ &        0.00$^*$ \\
2 &       + MinOut (k=2) &         0.85$^*$ &            0.11\hspace{0.5em} &           0.32$^*$ &       0.02$^*$ &        0.00\hspace{0.5em} \\
1 &   + MaxFit (L2=0.01) &         0.86$^*$ &            0.09$^*$ &           0.31\hspace{0.5em} &       0.02\hspace{0.5em} &        0.00\hspace{0.5em} \\
5 &          + Quadratic &         0.87$^*$ &            0.08$^*$ & 